# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [3]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd

# To process large dataset
import dask
import dask.dataframe as dd

# To check performance of the operation
import time

from matplotlib_venn import venn2
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import Imputer, StandardScaler

# magic word for producing visualizations in notebook
%matplotlib inline

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

### Load Data
Since the dataset is more than 1GB, I've used `c` engine instead of `python` to load data faster. In addition, there are mixed data in column 18 & 19; more specifically some `NaN` values are represented by `X` & `XX`. These values are set as `na_values`.

In [ ]:
%%time

# Demographics data for the general population of Germany
azdias = pd.read_csv('Udacity_AZDIAS_052018.csv', na_values=["X", "XX"], engine="c")

# Demographics data for customers of a mail-order company
customers = pd.read_csv('Udacity_CUSTOMERS_052018.csv', na_values=["X", "XX"], engine="c")

#### Display Preliminary Information about the Dataframes

In [ ]:
azdias.head()

In [ ]:
azdias.info(verbose=True)

In [ ]:
azdias.select_dtypes(include="object")

In [ ]:
customers.head()

In [ ]:
# There are three features in 'customers' which are not present in 'azdias'. These features are redundant.
customers = customers.drop(columns=["CUSTOMER_GROUP", "ONLINE_PURCHASE", "PRODUCT_GROUP"])

In [ ]:
customers.select_dtypes(include="object")

In [ ]:
customers.head()

In [ ]:
customers.info(verbose=True)

In [ ]:
# Find if both azdias and customers have same features
azdias_features = list(azdias.columns)
customers_feature = list(customers.columns)
set(azdias_features) == set(customers_feature)

#### Explore Features and Preprocess Data

In [ ]:
# Load .xlsx file for Attributes description
# Skip first row and slice the dataframe to drop first column which are empty cells in excel
attributes = pd.read_excel("DIAS Attributes - Values 2017.xlsx", header=1)
attributes = attributes.loc[:,["Attribute", "Value", "Meaning"]].fillna(method='ffill')
attributes.head()

In [ ]:
# List of unique attributes in 'atrtibutes'
attribute_list = list(attributes.Attribute.unique())
len(attribute_list)

**Observations:** Both `azdias` & `customers` have 366 features while the `attributes` only contains 314 features. The missing or unknown values in the features from `azdias` & `customers` that are also present in `attributes`, can be referred from `attributes` and mapped to `NaN`.

In [ ]:
# Find common and unique features between 'azdias' and 'attributes'
common_features = set(azdias_features) & set(attribute_list)
unique_azdias_features = set(azdias_features) - set(attribute_list)
num_common_features = len(common_features)
num_unique_azdias_features = len(unique_azdias_features)
print("common features: {}, unique azdias features: {}".format(num_common_features, num_unique_azdias_features))

In [ ]:
# Select the features whose attributes are known
azdias_subset = azdias.drop(columns=list(unique_azdias_features))

In [ ]:
azdias_subset.shape

**Observation:** `azdias` & `customers` have 272 common features with `attributes`. The features with unknown or no information value will be considered as `NaN`. The steps to do so are listed below:
1. From `attributes` _unknown_ or _no information_ values are identified  
2. Subset of `attributes` with _unknown_ or _missing values_ was created 
3. Check if the values in `azdias` or `customers` match with _unknown_ or _missing values_
4. The unknown values in `attributes` are assigned `NaN`

In [ ]:
# Create a subset of attributes with unknown or no information value
attributes_unknown_val = attributes[(attributes['Meaning'].str.contains("unknown") | \
                                 attributes['Meaning'].str.contains("no "))]
attributes_unknown_val.head()

For some attributes, the unknown or no information values are list while for other attributes they are integer value. Unknown values for each attributes will be converted to list for consistency.

In [ ]:
%%time
unknown_val = []
for attribute in attributes_unknown_val['Attribute'].unique():
    val = attributes_unknown_val.loc[attributes['Attribute'] == attribute, 'Value'].astype("str") \
                                                                        .str.cat(sep=',').split(',')
    val = list(map(int, val))
    unknown_val.append(val)
attributes_unknown_val = pd.concat([pd.Series(attributes_unknown_val['Attribute'].unique()),
                                    pd.Series(unknown_val)], axis=1)

attributes_unknown_val.columns = ['attribute', 'unknown']
attributes_unknown_val.head()

In [ ]:
%%time
# We know which features are common between azdias and attributes. So we will replace the unknown or 
# missing information with NaN    
for row in attributes_unknown_val.itertuples(index=False):
    if row.attribute in azdias_subset.columns.values.tolist():
        print(row.attribute)
        nan_val = attributes_unknown_val.loc[attributes_unknown_val['attribute'] == row.attribute, 'unknown'].iloc[0]
        #print(nan_val)
        #na_map = list(map(str.strip, na_map.strip('][').replace("'", '').split(',')))
        nan_idx = azdias_subset.loc[:, row.attribute].isin(nan_val)
        #print(nan_idx)
        azdias_subset.loc[nan_idx, row.attribute] = np.NaN
    else:
        continue

In [ ]:
azdias_subset.shape

```
start = time.time()
    
for row in attributes_unknown_val.itertuples(index=False):
    if row.attribute in customers.columns.values.tolist():
        print(row.attribute)
        nan_val = attributes_unknown_val.loc[attributes_unknown_val['attribute'] == row.attribute, 'unknown'].iloc[0]
        #print(nan_val)
        #na_map = list(map(str.strip, na_map.strip('][').replace("'", '').split(',')))
        nan_idx = customers.loc[:, row.attribute].isin(nan_val)
        #print(nan_idx)
        customers.loc[nan_idx, row.attribute] = np.NaN
    else:
        continue        
end = time.time()

customers.head(20)
```

In [ ]:
# Total number of NaN count on each column
nan_count_column_azdias = azdias_subset.isnull().sum()
#nan_count_column_azdias.sort_values(ascending=False)

# Plot the distribution of missing or unknown data for each column
plt.figure(figsize=(15,5))
plt.title('Distribution of Missing Data in Each Column')
plt.hist(nan_count_column_azdias, bins=100)
plt.ylabel('Number of Columns')
plt.xlabel('Number of Missing Values')
plt.show()

**Observations:** There are 891221 rows in the `azdias` dataset. If more than 200000 rows have missing values in a column, it indicates two or more rows are missing for every five rows. It will be hard to extract any significant information from these columns. Therefore, these columns will be dropped. Similar approach will be taken for rows as well.

In [ ]:
# Select the columns that are needed to be dropped
drop_columns = nan_count_column_azdias[nan_count_column_azdias > 200000]

# Make list of the drop_columns
drop_column_list = drop_columns.index.tolist()

# Drop the columns from azdias
azdias_subset.drop(columns=drop_column_list, inplace=True)

In [ ]:
azdias_subset.shape

In [ ]:
# Select the rows that are needed to be dropped
count_nan_row = azdias_subset.isnull().sum(axis=1)
plt.figure(figsize=(15,5))
plt.title('Distribution of Missing Data in Each Row')
plt.hist(count_nan_row, bins=50)
plt.ylabel('Number of Rows')
plt.xlabel('Number of Missing Values')
plt.show()

In [ ]:
# Dropping rows where more than 50 values are missing or unknown
count_nan_row = azdias_subset.shape[1] - azdias_subset.count(axis=1)
drop_row = azdias_subset.index[count_nan_row > 50]
azdias_subset.drop(drop_row, axis=0, inplace=True)

In [ ]:
azdias_subset.shape

In [ ]:
azdias_subset.to_csv("azdias_subset_clean.csv", index=False)

In [ ]:
azdias_subset = pd.read_csv("azdias_subset_clean.csv")
azdias_subset.head()

### Encode Features

In [ ]:
categorical = ['AGER_TYP', 'ANREDE_KZ', 'BIP_FLAG', 'CAMEO_DEU_2015', 'CAMEO_DEUINTL_2015', 'CJT_GESAMTTYP',
'D19_KONSUMTYP', 'D19_KK_KUNDENTYP', 'FINANZTYP', 'GEBAEUDETYP', 'GEBAEUDETYP_RASTER', 'GFK_URLAUBERTYP',
'HAUSHALTSSTRUKTUR', 'HEALTH_TYP', 'KBA05_HERSTTEMP', 'KBA05_MAXHERST', 'KBA05_MODTEMP', 'LP_FAMILIE_FEIN',
'LP_FAMILIE_GROB', 'LP_LEBENSPHASE_FEIN', 'LP_LEBENSPHASE_GROB', 'LP_STATUS_FEIN', 'LP_STATUS_GROB',
'NATIONALITAET_KZ', 'OST_WEST_KZ', 'PRAEGENDE_JUGENDJAHRE', 'SHOPPER_TYP', 'SOHO_FLAG', 'TITEL_KZ', 'VERS_TYP']

In [ ]:
%%time
# let's go through categorial columns and check what unique values they have in each columns
for i in categorical:
    if i in azdias_subset.columns:
        print(i, len(azdias_subset[i].value_counts()), azdias_subset[i].unique().tolist())
    else:
        print(i + " dropped already")

In [ ]:
azdias_subset["OST_WEST_KZ"].head(50)

In [ ]:
# replacing characters with 1 and 0
azdias_subset['OST_WEST_KZ'].replace(['W', 'O'], [1.0, 0.0], inplace=True)

In [ ]:
azdias_subset['OST_WEST_KZ'].head()

In [ ]:
# drop fine features, rough features are still available
azdias_subset.drop(['CAMEO_DEU_2015', 'LP_FAMILIE_FEIN', 'LP_STATUS_FEIN'], axis=1, inplace=True)

In [ ]:
azdias_subset.head()

In [ ]:
azdias_subset.info(verbose=True)

In [ ]:
azdias_subset[["GEBURTSJAHR", "GREEN_AVANTGARDE", "PRAEGENDE_JUGENDJAHRE"]].head()

In [ ]:
def convert_pj_to_dec(val):
    """
    Converts value of feature PRAEGENDE_JUGENDJAHRE to a DECADE value.

    INPUT:
    - val (int): original value

    OUTPUT:
    - int: converted value (0: 40s, 1: 50s, 2: 60s, 3: 70s, 4: 80s, 5: 90s)
    """
    
    result = val
    if (val > 0 and val <= 2):
        result = 0 #40s
    elif (val <= 4):
        result = 1 #50s
    elif (val <= 7):
        result = 2 #60s
    elif (val <= 9):
        result = 3 #70s
    elif (val <= 13):
        result = 4 #80s
    elif (val <= 15):
        result = 5 #90
    return result

In [ ]:
def convert_pj_to_move(val):
    """
    Converts value of feature PRAEGENDE_JUGENDJAHRE to a MOVEMENT value.

    INPUT:
    - val (int): original value

    OUTPUT:
    - int: converted value (0: Mainstream, 1: Avantgarde)
    """
    
    result = val
    if (val in [1,3,5,8,10,12,14]):
        result = 0 # M
    elif (val in [2,4,6,7,9,11,13,15]):
        result = 1 # A
    return result

In [ ]:
# Investigate "PRAEGENDE_JUGENDJAHRE" and engineer two new variables.
azdias_subset['DECADE'] = azdias_subset['PRAEGENDE_JUGENDJAHRE'].apply(lambda x: convert_pj_to_dec(x))
azdias_subset['MOVEMENT'] = azdias_subset['PRAEGENDE_JUGENDJAHRE'].apply(lambda x: convert_pj_to_move(x))

# drop PRAEGENDE_JUGENDJAHRE
azdias_subset.drop('PRAEGENDE_JUGENDJAHRE', axis=1, inplace=True)

In [ ]:
# drop LP_LEBENSPHASE_FEIN and LP_LEBENSPHASE_GROB because no clear structure available
azdias_subset.drop(['LP_LEBENSPHASE_FEIN', 'LP_LEBENSPHASE_GROB'], axis=1, inplace=True)

In [ ]:
azdias_subset.head()

In [ ]:
azdias_subset.shape

#### Feature Engineering

In [ ]:
imputer = Imputer(strategy='median')
scaler = StandardScaler()

# impute median and scale azdias
azdias_imputed = pd.DataFrame(imputer.fit_transform(azdias_subset))
azdias_scaled = scaler.fit_transform(azdias_imputed)

In [4]:
def load_data(azdias_filepath, customers_filepath, attributes_filepath):
    """
    Method for loading dataset from CSV & Excel
    
    Args:
        azdias_filepath (str): Azdias Filepath
        customers_filepath (str): Customers Filepath
        attributes_filepath (str): Attributes Filepath
        
    Output:
        azdias: Pandas Dataframe
        customers: Pandas Dataframe
        attributes: Pandas Dataframe
    """
    
    # Load "azdias" dataset
    azdias = pd.read_csv(azdias_filepath, na_values=["X", "XX"], engine="c")
    
    # Load "customers" dataset
    customers = pd.read_csv(customers_filepath, na_values=["X", "XX"], engine="c")
    
    # Load "attributes" dataset
    attributes = pd.read_excel("DIAS Attributes - Values 2017.xlsx", header=1).loc[:, ["Attribute", "Value", "Meaning"]] \
                                .fillna(method='ffill')
    
    return azdias, customers, attributes

In [5]:
azdias_filepath = "Udacity_AZDIAS_052018.csv"
customers_filepath = "Udacity_CUSTOMERS_052018.csv"
attributes_filepath = "DIAS Attributes - Values 2017.xlsx"

azdias, customers, attributes = load_data(azdias_filepath, customers_filepath, attributes_filepath)

In [6]:
attributes.head()

,Attribute,Value,Meaning
0,AGER_TYP,-1,unknown
1,AGER_TYP,0,no classification possible
2,AGER_TYP,1,passive elderly
3,AGER_TYP,2,cultural elderly
4,AGER_TYP,3,experience-driven elderly


In [17]:
def clean_data(df, attributes, col_nan_threshold, row_nan_threshold):
    """
    Method for cleaning the dataset according to attributes
    
    Args:
        df (Pandas Dataframe): Dataset to clean
        attributes (Pandas Dataframe): Reference dataset for cleaning df
        col_nan_threshold (float): Threshold value (0-1) for missing NaN count in a column
        
    Output:
        df (Pandas Dataframe): Cleaned dataset
    """
    
    # List of attributes
    attribute_list = list(attributes.Attribute.unique())
    
    # List of df attributes
    df_attributes = list(df.columns)
    
    # Find attributes that are unique in df
    unique_df_attributes = list(set(df_attributes) - set(attribute_list))    
    
    # Drop attributes from df that are not present in attribute
    df.drop(columns=unique_df_attributes, inplace=True)
    
    # Create a subset of attributes with unknown or no information value
    unknown_val = []
    attributes_unknown_val = attributes[(attributes['Meaning'].str.contains("unknown") | 
                                         attributes['Meaning'].str.contains("no "))]
    for attribute in attributes_unknown_val['Attribute'].unique():
        val = attributes_unknown_val.loc[attributes['Attribute'] == attribute, 'Value'].astype("str") \
                                                                            .str.cat(sep=',').split(',')
        val = list(map(int, val))
        unknown_val.append(val)
    attributes_unknown_val = pd.concat([pd.Series(attributes_unknown_val['Attribute'].unique()),
                                        pd.Series(unknown_val)], axis=1)
    attributes_unknown_val.columns = ['attribute', 'unknown']
    
    # Replace unknown or missing values in df with NaN   
    for row in attributes_unknown_val.itertuples(index=False):
        if row.attribute in df.columns.values.tolist():
            nan_val = attributes_unknown_val.loc[attributes_unknown_val['attribute'] == row.attribute, 'unknown'].iloc[0]
            nan_idx = df.loc[:, row.attribute].isin(nan_val)
            df.loc[nan_idx, row.attribute] = np.NaN
        else:
            continue
    
    # Drop columns with NaN count above the threshold
    nan_count_column = (df.isnull().sum()/df.shape[0]).sort_values(ascending=False)
    # Select the columns that are needed to be dropped
    nan_count_column = nan_count_column[nan_count_column > col_nan_threshold]
    # Make list of the drop_columns
    drop_column_list = nan_count_column.index.tolist()
    # Drop the columns from azdias
    df.drop(columns=drop_column_list, inplace=True)
    
    # Dropping rows where more than 50 values are missing or unknown
    number_col = df.shape[1]
    count_nan_row = (df.shape[1] - df.count(axis=1)) / number_col
    drop_row = df.index[count_nan_row > row_nan_threshold]
    df.drop(drop_row, axis=0, inplace=True)
    
    return df

In [19]:
df = clean_data(azdias, attributes, 0.2, 0.2)
df.head()

,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_PERSONEN,ANZ_TITEL,BALLRAUM,CAMEO_DEU_2015,CAMEO_DEUG_2015,CJT_GESAMTTYP,EWDICHTE,FINANZ_ANLEGER,...,SEMIO_TRADV,SEMIO_VERT,SHOPPER_TYP,VERS_TYP,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
1,11.0,0.0,2.0,0.0,6.0,8A,8.0,5.0,3.0,5.0,...,6.0,1.0,3.0,2.0,3.0,9.0,4.0,5.0,2.0,1.0
2,10.0,0.0,1.0,0.0,2.0,4C,4.0,3.0,4.0,2.0,...,3.0,4.0,2.0,1.0,3.0,9.0,2.0,5.0,2.0,3.0
3,1.0,0.0,0.0,0.0,4.0,2A,2.0,2.0,2.0,2.0,...,4.0,4.0,1.0,1.0,NaN,9.0,7.0,3.0,2.0,4.0
4,3.0,0.0,4.0,0.0,2.0,6B,6.0,5.0,5.0,1.0,...,2.0,7.0,2.0,2.0,2.0,9.0,3.0,4.0,1.0,3.0
5,5.0,0.0,1.0,0.0,6.0,8C,8.0,2.0,2.0,2.0,...,6.0,2.0,0.0,2.0,6.0,9.0,7.0,4.0,2.0,1.0


In [26]:
# List of categorical variable
category_list = ['ANREDE_KZ', 'CJT_GESAMTTYP', 'FINANZTYP', 'GFK_URLAUBERTYP', 'GREEN_AVANTGARDE', 'LP_FAMILIE_FEIN',
                 'LP_FAMILIE_GROB', 'LP_STATUS_FEIN', 'LP_STATUS_GROB', 'NATIONALITAET_KZ', 'SHOPPER_TYP', 'SOHO_KZ',
                 'VERS_TYP', 'ZABEOTYP', 'GEBAEUDETYP', 'OST_WEST_KZ', 'CAMEO_DEUG_2015', 'CAMEO_DEU_2015', 'D19_KONSUMTYP',
                 'KBA05_MAXHERST', 'KBA05_MAXSEG']
len(category_list)

21

In [34]:
azdias.to_csv("test.csv", index=False)

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
#mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter. If you're one of the top performers, you may have the chance to be contacted by a hiring manager from Arvato or Bertelsmann for an interview!

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [ ]:
#mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';')